## Problema

El problema consiste en predecir la probabilidad de atraso de los vuelos que aterrizan o despegan del aeropuerto de Santiago
de Chile (SCL). Para eso les entregamos un dataset usando datos públicos y reales donde cada fila corresponde a un vuelo
que aterrizó o despegó de SCL. Para cada vuelo se cuenta con la siguiente información:

* **Fecha-I** : Fecha y hora programada del vuelo.
* **Vlo-I** : Número de vuelo programado.
* **Ori-I** : Código de ciudad de origen programado.
* **Des-I** : Código de ciudad de destino programado.
* **Emp-I** : Código aerolínea de vuelo programado.
* **Fecha-O** : Fecha y hora de operación del vuelo.
* **Vlo-O** : Número de vuelo de operación del vuelo.
* **Ori-O** : Código de ciudad de origen de operación
* **Des-O** : Código de ciudad de destino de operación.
* **Emp-O** : Código aerolínea de vuelo operado.
* **DIA** : Día del mes de operación del vuelo.
* **MES** : Número de mes de operación del vuelo.
* **AÑO** : Año de operación del vuelo.
* **DIANOM** : Día de la semana de operación del vuelo.
* **TIPOVUELO** : Tipo de vuelo, I =Internacional, N =Nacional.
* **OPERA** : Nombre de aerolínea que opera.
* **SIGLAORI** : Nombre ciudad origen.
* **SIGLADES** : Nombre ciudad destino.

## 1. Cargar Librerías

In [13]:
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from xgboost import plot_importance
from sklearn.utils import resample
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import seaborn as sn
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import LabelEncoder

## 2. Definición de Columnas Basales

In [14]:
COLUMNAS_BASALES = ["OPERA", "TIPOVUELO", "MES"]
model_dictionary = {}

## 3. Creación de Funciones

In [26]:
# funcion de upsampling realizada en notebook de Datascientist (implemenatada por completitud)
def upsample_data(X, Y):
    index_puntual = Y == 0
    index_retraso = np.logical_not(index_puntual)
    data_no_retraso = X[index_puntual]
    data_atraso = X[index_retraso]

    data_atraso_upsampled = resample(data_atraso, 
                                    replace = True,     
                                    n_samples = int(0.35*X.shape[0]))
    Y1 = resample(Y[index_retraso],
                  replace = True,     
                    n_samples = int(0.35*X.shape[0]))
    
    Y0 = Y[index_puntual]
    data_upsampled = pd.concat([data_no_retraso, data_atraso_upsampled])
    Y = pd.concat([Y0, Y1])
    return data_upsampled, Y


def prepare_categorical_data(data, columns):

    ## VARIABLES:
    # data [Dataframe]: El Dataframe donde está la data
    # columns: nombre de las columnas que deben ser transformadas a variables dummy

    # funcion que se utiliza para preparar los datos para que sean consumidos por el modelo 
    categorical_transformed_columns = [pd.get_dummies(data[a_column], prefix = a_column) for a_column in columns]
    features = pd.concat(categorical_transformed_columns, axis = 1)
    #enc = LabelEncoder()
    #features = data.loc[:, columns].apply(enc.fit_transform)
    return features

def generate_splitted_XY(data, columns=COLUMNAS_BASALES, stratify=False): 
    # funcion que genera los conjuntos de entrenamiento y testeo: 
    # data [DataFrame]: los datos que tenemos q disposicion.
    # columns: las columnas que queremos tranformar a dummy
    # stratify: permite hacer los conjuntos de testeo y validación de una manera estratificada. 
    X, Y = divide_XY(data, columns)
    train_test_kwargs = {}
    if stratify:
        train_test_kwargs.update({ "stratify": Y})
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42, **train_test_kwargs)
    return x_train, x_test, y_train, y_test, X, Y

def divide_XY(data, columns=COLUMNAS_BASALES):
    X = prepare_categorical_data(data, columns)
    Y = data["atraso_15"]
    return X, Y
    

<!-- ## 4. Cargar datos -->

## 4. Cargar datos

In [147]:
# IMPORTAMOS LOS DATOS DE LA BASE DE DATOS ORIGINAL
df_real = pd.read_csv("Datasets/dataset_SCL.csv", low_memory=False)
# IMPORTAMOS LAS FEATURES SINTETICAS
df_synthetic = pd.read_csv("Datasets/synthetic_features.csv", low_memory=False)
# UNIMOS AMBOS DATAFRAMES
df = pd.concat([df_real, df_synthetic], axis=1)
# LOS MOSTRMOS:
df.head()

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES,temporada_alta,dif_min,atraso_15,periodo_dia
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,...,2017,Domingo,I,American Airlines,Santiago,Miami,1,3.0,0,noche
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,...,2017,Lunes,I,American Airlines,Santiago,Miami,1,9.0,0,noche
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,...,2017,Martes,I,American Airlines,Santiago,Miami,1,9.0,0,noche
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,...,2017,Miercoles,I,American Airlines,Santiago,Miami,1,3.0,0,noche
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,...,2017,Jueves,I,American Airlines,Santiago,Miami,1,-2.0,0,noche


In [149]:
data = shuffle(df[['OPERA', 'MES', 'TIPOVUELO', 'SIGLADES', 'SIGLAORI', 'DIANOM', 'atraso_15']], random_state = 111)

In [29]:
Xo, Yo = divide_XY(data)
_, xo_test, _, yo_test = train_test_split(Xo, Yo, test_size = 0.33, random_state = 42, stratify=Yo)

In [30]:
print(f"Cantidad de vuelos {Yo.count()}",f"Cantidad de atrasos {Yo.sum()}", f"Porcentaje de vuelos atrasados en dataset:{Yo.sum()/Yo.count()}")
print(f"Cantidad de vuelos train {yo_test.count()}", f"Cantidad de atrasos {yo_test.sum()}", f"Porcentaje de vuelos atrasados en dataset:{yo_test.sum()/yo_test.count()}")

Cantidad de vuelos 68206 Cantidad de atrasos 12614 Porcentaje de vuelos atrasados en dataset:0.18493974137172683
Cantidad de vuelos train 22508 Cantidad de atrasos 4163 Porcentaje de vuelos atrasados en dataset:0.18495645992535986


# 4.1 Pregunta 1:
Escoge el modelo que a tu criterio tenga un mejor performance, argumentando tu decisión.


**Respuesta:** 
La pregunta sobre que modelo es mejor, tiene directa relación con el problema a resolver. Como se indica en el enunciado del Jupyter notebook de Juan, el problema definido es el de predecir el retraso de aviones. De esta manera, a la hora de saber que modelo es mejor es necesario comparar la métrica que mejor represente el comportamiento que queremos del modelo. De este modo, a continuación y a modo de resumen adjunto los resultados obtenidos por los modelos entrenados en el notebook de Juan. 

### Modelo de Regresión Logística: 
|   | precision | recall | f1-score | support |
|---|-----------|--------|----------|---------|
| 0 | 0.82      | 1      | 0.9      | 18403   |
| 1 | 0.57      | 0.03   | 0.06     | 4105    |
### Modelo XGBoost
|   | precision | recall | f1-score | support |
|---|-----------|--------|----------|---------|
| 0 | 0.82      | 1      | 0.9      | 18403   |
| 1 | 0.67      | 0.02   | 0.04     | 4105    |
### Modelo XGBoost con mejores Features:
|   | precision | recall | f1-score | support |
|---|-----------|--------|----------|---------|
| 0 | 0.82      | 1      | 0.9      | 18403   |
| 1 | 0.66      | 0.01   | 0.03     | 4105    |
### Modelo XGBoost con upsampled data 
|   | precision | recall | f1-score | support |
|---|-----------|--------|----------|---------|
| 0 | 0.68      | 0.93   | 0.79     | 18403   |
| 1 | 0.62      | 0.19   | 0.3      | 9897    |

La predicción de retrasos en aviones para las aerolines tiene beneficios no solamente para los pasajeros, sino que para la aerolínea también. Desde un punto de vista puramente de costos operacionales según <a href=https://www.airlines.org/dataset/u-s-passenger-carrier-delay-costs>Airlines for America</a> el costo por minuto de "Block time" ronda los $80 dólares (cifras 2021), este tiempo considera el minuto desde que el avión cierra sus puertas hasta el instante en que este llega a la compuerta en su destino. Considerando lo anterior un avión que sufra retrasos tendrá un costo tanto por su tiempo de vuelo como además el tiempo que utilice para llegar a la compuerta. 

Además del costo anterior, existe un costo económico que debe ser considerado, ya que según <a href=https://www.sernac.cl/portal/607/w3-article-5594.html>Sernac</a> los pasajeros que sufran por retrasos en sus vuelos tienen, en algunos casos, derecho a una indemnización. Esto sumado a la entrega de una mala experiencia para los usuarios que es fundamental para cualquier aerolínes, ya que parte importante de la competencia existente entre ellas es tanto el precio como la experiencia de viaje que puedan proporcionarle al usuario. 

Debido a esto, al momento de desarrollar un modelo que realice predicciones sobre el retraso de los vuelos de aviones, la consideración más importante es la cantidad de vuelos que efectivamente llegaron tarde y fueron seleccionados por el modelo. Tener seguridad de que el modelo presentado tiene una alta taza de recall sobre la clase de retraso es importante ya que permite que la aerolínea al saber sobre dicho retraso pueda tomar acción desde el punto de vista de la gestión de tanto el vuelo como del pasajero, además es importante que la tasa de precisión no sea baja ya que también es importante que la detección de atrasos no tenga falsos positivos.  

De este modo, si se consideran los modelos entrenados en el notebook de Juan, se podrá ver que el modelo de recall con upsampling tiene el mayor recall de todos, pero al momento de hacer dicho aumento de muestras dicho aumento fue realizado también sobre la clase de prueba, por lo que los resultados encontrados en esa corrida del algoritmo están sesgados a que la distribución de clases presentes en los datos de testeo no son los originales. Debido a esto es necesario correr nuevamente dicho algoritmo.

## 5. Muestra original de los  datos

### 5.1 Modelo Basal

<ol>
  <p>Para este modelo las columnas que se utilizarán serán las mismas utilizadas en el notebook anterior. Dichas columnas son las siguientes: </p>
  <li>OPERA</li>
  <li>TIPOVUELO</li>
  <li>MES</li>
</ol>

Un punto importante a considerar en el notebook, es que al momento de extraer los datos se utilizan las siguientes columnas:
'OPERA', 'MES', 'TIPOVUELO', 'SIGLADES', 'DIANOM', 'atraso_15'

Pero al momento de generar los datos para entrenar los modelos no son utilizadas todas las columnas. Por esto, el modelo que se genera solamente trabajará con: 'OPERA', 'MES', 'TIPOVUELO'. Lo anterior se hace para poder entrenar el modelo basal. 

A continuación, se muestra la linea de la celda textual (sección 4.) que genera los datos para los modelos de predicción:

features = pd.concat([pd.get_dummies(data['OPERA'], prefix = 'OPERA'),pd.get_dummies(data['TIPOVUELO'], prefix = 'TIPOVUELO'), pd.get_dummies(data['MES'], prefix = 'MES')], axis = 1)





In [31]:
x_train, x_val, y_train, y_val, X, Y = generate_splitted_XY(data, stratify=False)

In [32]:
print(f"Cantidad de vuelos {Y.count()}",f"Cantidad de atrasos {Y.sum()}", f"Porcentaje de vuelos atrasados en dataset:{Y.sum()/Y.count()}")
print(f"Cantidad de vuelos train {y_train.count()}", f"Cantidad de atrasos {y_train.sum()}", f"Porcentaje de vuelos atrasados en dataset:{y_train.sum()/y_train.count()}")
print(f"Cantidad de vuelos train {y_val.count()}", f"Cantidad de atrasos {y_val.sum()}", f"Porcentaje de vuelos atrasados en dataset:{y_val.sum()/y_val.count()}")

Cantidad de vuelos 68206 Cantidad de atrasos 12614 Porcentaje de vuelos atrasados en dataset:0.18493974137172683
Cantidad de vuelos train 45698 Cantidad de atrasos 8509 Porcentaje de vuelos atrasados en dataset:0.18620070900258218
Cantidad de vuelos train 22508 Cantidad de atrasos 4105 Porcentaje de vuelos atrasados en dataset:0.18237959836502576


In [33]:
x_train, y_train = upsample_data(x_train, y_train)

In [34]:
print(f"Cantidad de vuelos {Y.count()}",f"Cantidad de atrasos {Y.sum()}", f"Porcentaje de vuelos atrasados en dataset:{Y.sum()/Y.count()}")
print(f"Cantidad de vuelos train {y_train.count()}", f"Cantidad de atrasos {y_train.sum()}", f"Porcentaje de vuelos atrasados en dataset:{y_train.sum()/y_train.count()}")
print(f"Cantidad de vuelos train {y_val.count()}", f"Cantidad de atrasos {y_val.sum()}", f"Porcentaje de vuelos atrasados en dataset:{y_val.sum()/y_val.count()}")

Cantidad de vuelos 68206 Cantidad de atrasos 12614 Porcentaje de vuelos atrasados en dataset:0.18493974137172683
Cantidad de vuelos train 53183 Cantidad de atrasos 15994 Porcentaje de vuelos atrasados en dataset:0.30073519733749504
Cantidad de vuelos train 22508 Cantidad de atrasos 4105 Porcentaje de vuelos atrasados en dataset:0.18237959836502576


In [35]:
modelxgb_base = xgb.XGBClassifier(random_state=1,  objective= 'binary:logistic', learning_rate=0.01, subsample = 1, max_depth = 10)

In [36]:
modelxgb_base = modelxgb_base.fit(x_train, y_train,verbose=False)
predicted_xgb = modelxgb_base.predict(x_val)
print(classification_report(y_val, predicted_xgb, target_names=["no atrasado", "atrasado"]))

              precision    recall  f1-score   support

 no atrasado       0.83      0.98      0.90     18403
    atrasado       0.51      0.07      0.13      4105

    accuracy                           0.82     22508
   macro avg       0.67      0.53      0.51     22508
weighted avg       0.77      0.82      0.76     22508



In [37]:
predicted_xgb = modelxgb_base.predict(xo_test)
print(classification_report(yo_test, predicted_xgb, target_names=["no atrasado", "atrasado"]))

              precision    recall  f1-score   support

 no atrasado       0.82      0.98      0.90     18345
    atrasado       0.51      0.07      0.13      4163

    accuracy                           0.82     22508
   macro avg       0.67      0.53      0.51     22508
weighted avg       0.77      0.82      0.75     22508



In [38]:
model_dictionary["modelxgb_base"]= average_precision_score(yo_test, predicted_xgb)

In [39]:
model_dictionary

{'modelxgb_base': 0.20839491452833558}

Como se puede observar efectivamente los resultados del algoritmo con oversampling cambiaron bastante cuando se considera la distribución original de los datos. A pesar de eso, el modelo generado presenta mejores cifras que los modelos que los otros modelos generados por ende, el modelo seleccionado es el modelo XGBoost con oversampling. 

## 5.1.2Pregunta 2: 
Implementa mejoras sobre el modelo escogiendo la o las técnicas que prefieras

### Respuesta: 
Para mejorar el modelo es necesario considerar en primera instancia que clase de modelo estamos generando y si dicho modelo es el correcto para resolver el problema, además de estos es necesario considerar las funciones objetivo que dicho modelo utiliza para asegurarnos que estas funciones objetivo generan el efecto que se está buscando en este proyecto.
De esta manera, se puede descartar que el error de dicha implementación sea el uso de XGBoost, esto se debe a que este modelo es de los mejores modelos utilizados en la página kaggle, en específico en una gran cantidad de problemas de clasificación tanto multiclase como binario. De esta manera, aunque el tipo de modelo que se está utilizando no debe ser el problema talvés la manera en que se está inicializando y evaluando dicho modelo puede tener cierto espacio de mejora. 

Una de las primeras mejoras propuestas es el uso de un conjunto de validación que le permita al algoritmo ajustarse adecuadamente a la distribución real de los datos. Dicho conjunto de validación no se verá afectado por el efecto del cambio de distribución de los datos, permitiendo de este modo que el modelo sea fidedigno con la distribución original de los datos. Además, se define que la métrica a utilizar es el AUCPR (Área bajo la curva de Precisión-recall) los modelos que se evalúan con las curvas ROC sobre conjuntos de datos desbalanceados, tienden a sufrir de sobre ajusta con la clase mayoritaria, ya que las curvas roc contemplan en su cálculo la tasa de Verdaderos Negativos, que en caso de conjuntos de datos desbalanceados puede ser muy alta. Las curvas Precisión-recall por el otro lado no tienen esta dependencia y por ende no tienen dicho problema. 

Otra mejora que se implementará será cambiar el tipo de codificación de los datos de one hot encoding a un encoding ordinario, la razón de dicho cambio se debe al algoritmo que se está utilizando, como los algoritmos con árboles de decisión hacen particiones sobre los features, es más fácil para dicho algoritmo separar menos features en comparación a más. 

De esta manera, para generar mejoras sobre el modelo, en primera instancia se obtendrán los datos generados en el jupyter notebook de Juan y se correrá el modelo con dichos datos, de esta manera, los pasos con los que se intentará mejorar dicho modelo son los siguientes: 
<ol>
  <p>Con los features obtenidos por Juan:</p>
  <li>Se correrá el modelo basal con los datos entregados para tener un punto de comparación</li>
  <li>Se hará un grid search sobre nuevos parámetros que se le agreguen al modelo para mejorar su desempeño</li>
  <li>Se realizará oversampling utilizando métodos más complejos de oversampling como es SMOTE</li>
  <li>Se realizará oversampling con SMOTE y random undersampling (siendo este la manera recomendada de corre dicho algoritmo)</li>
  <li>Grid Search sobre los parámetros del modelo</li>

  <p>Uso de otros features:</p>
  <li>Se Utilizarán más features para así enriquecer el modelo generado</li>
  <li>Se obtendrán resultados con los mismos métodos de over y under sampling anteriormente mencionados</li>
  <li>Grid Search sobre los parámetros del modelo</li>
</ol>

En cada una de las dos etapas propuestas se generará un conjunto de test el cual viene directamente de los datos muestreados y no presenta modificaciones con respecto a las proporciones originales que presenta dicho dataset. 

### 5.2 Grid Search con parámetros de regularización:
A modo de conjetura, se cree que el modelo al tener una  profundidad máxima de 10 puede estar sufriendo de sobre ajuste. Además, el modelo de XGBoost utiliza una regularización $\alpha$ para regularizar mediante la norma $L_{2}$, dicho modelo también tiene otras formas de evitar el sobre ajuste y una posible mejora para el modelo puede ser ajustar dicho parámetro, ya que permite que el árbol sufra en menor medida de sobre ajuste debido a la penalización.   

Además, para encontrar el mejor modelo posible lo que haremos será agregar un conjunto de validación el cual se utilizará para hacer la selección del modelo. Como se dijo anteriormente el resultado sobre los datos originales sin cambios en sus procentajes definirá cuál será el modelo que utilizaremos. 

De este modo, lo que se propone es variar los hiperparámetros que generen mejoras en el sobre ajuste del modelo. En específico, estos parámetros son:
  - $\alpha$
  - $\lambda$
  - Profundidad máxima
  - scale_pos_weight
  - subsample



generamos la nueva codificacion de features:

In [76]:
# obtenemos los datos del dataframe:
data_to_pipe = data[COLUMNAS_BASALES]

# volvemos a generar los datos originales con la nueva codificacion:
Xo, Yo = data_to_pipe, data["atraso_15"]
_, xo_test, _, yo_test = train_test_split(Xo, Yo, test_size = 0.33, random_state = 42, stratify=Yo)

In [77]:
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder
# generamos una codificaciones ordinaria para los datos: 
column_trans = make_column_transformer((OrdinalEncoder(), COLUMNAS_BASALES))
# ajustamos la codificacion
column_trans.fit(data_to_pipe)

# generamos una funcion que genere pipelines: 
Makepipe = lambda the_model: make_pipeline(column_trans, the_model)


In [78]:
# generamos un kfold stratificado para la búsqueda de los modelos: 
skf = StratifiedKFold(n_splits=2, random_state=42, shuffle=True)

In [79]:
x_train, x_val, y_train, y_val = train_test_split(Xo, Yo, test_size=0.33, random_state=42, shuffle=True)

In [80]:
x_train, y_train = upsample_data(x_train, y_train)

In [81]:
print(f"Cantidad de vuelos train {y_train.count()}", f"Cantidad de atrasos {y_train.sum()}", f"Porcentaje de vuelos atrasados en dataset:{y_train.sum()/y_train.count()}")
print(f"Cantidad de vuelos validacion {y_val.count()}", f"Cantidad de atrasos {y_val.sum()}", f"Porcentaje de vuelos atrasados en dataset:{y_val.sum()/y_val.count()}")

Cantidad de vuelos train 53183 Cantidad de atrasos 15994 Porcentaje de vuelos atrasados en dataset:0.30073519733749504
Cantidad de vuelos validacion 22508 Cantidad de atrasos 4105 Porcentaje de vuelos atrasados en dataset:0.18237959836502576


In [82]:
# para poder hacer correctamente el grid search se hace la codificacion de las entradas
x_train, x_val, xo_test= column_trans.transform(x_train), column_trans.transform(x_val), column_trans.transform(xo_test)

In [67]:
modelxgb_grid = xgb.XGBClassifier()
parameters = {"eta": [0.01],
    "scale_pos_weight": [1, 1.2, 3, 5],
    "reg_lambda": [0, 0.5, 1],
    "reg_alpha": [0, 0.5, 1],
    "gamma": [0, 0.25, 1],
    "subsample": [1],
    "max_depth": [10]}
xgb_grid_base = GridSearchCV(modelxgb_grid, param_grid=parameters,
    cv = 5, n_jobs=-1, verbose=1)


In [68]:
xgb_grid_base.fit(x_train, y_train, verbose=False)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'eta': [0.01], 'gamma': [0, 0.25, 1],
                         'max_depth': [10], 'reg_alpha': [0, 0.5, 1],
                         'reg_lambda': [0, 0.5, 1],
                         'scale_pos_weight': [1, 1.2, 3, 5], 'subsample': [1]},
             verbose=1)

In [83]:
predicted_xgb = xgb_grid_base.predict(x_val)
print(classification_report(y_val, predicted_xgb))


              precision    recall  f1-score   support

           0       0.83      0.97      0.90     18403
           1       0.48      0.11      0.17      4105

    accuracy                           0.82     22508
   macro avg       0.66      0.54      0.54     22508
weighted avg       0.77      0.82      0.76     22508



In [84]:
predicted_xgb = xgb_grid_base.predict(xo_test)
print(classification_report(yo_test, predicted_xgb))

              precision    recall  f1-score   support

           0       0.83      0.97      0.90     18345
           1       0.49      0.11      0.18      4163

    accuracy                           0.81     22508
   macro avg       0.66      0.54      0.54     22508
weighted avg       0.76      0.81      0.76     22508



In [85]:
model_dictionary["xgb_grid_base"]= average_precision_score(yo_test, predicted_xgb)

In [86]:
xgb_grid_base.best_params_

{'eta': 0.01,
 'gamma': 1,
 'max_depth': 10,
 'reg_alpha': 1,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'subsample': 1}

Ronda 2: 
Los parámetros que quedaron en posiciones extremas y que dichas posiciones no sean el final de su rango, serán re ajustadas nuevamente para ver si es posible encontrar un nuevo óptimo. 

In [87]:
modelxgb_grid_round2 = xgb.XGBClassifier(random_state=1, objective= 'binary:logistic', subsample=0.9, eval_metric='aucpr')
parameters = {"eta": [0.01],
    "scale_pos_weight": [1],
    "reg_lambda": [1, 1.5, 3],
    "reg_alpha": [1, 1.5, 3],
    "gamma": [1],
    "subsample": [1],
    "max_depth": [10]}
clf_grid_round2 = GridSearchCV(modelxgb_grid_round2, param_grid=parameters,
    cv = skf, n_jobs=-1, verbose=1)

In [88]:
clf_grid_round2.fit(x_train, y_train, eval_set=[(x_val, y_val)], verbose=False)

Fitting 2 folds for each of 9 candidates, totalling 18 fits


GridSearchCV(cv=StratifiedKFold(n_splits=2, random_state=42, shuffle=True),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='aucpr', feature_types=None,
                                     gamma=None, gpu_id=None, grow_policy=None,
                                     importance_...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=1, ...),
             n_jobs=-1,
             param_grid={'eta': [0.01], 'gamma': [1], 'max_depth': [10],
                         'reg_alpha': [1, 1.5, 3], 'reg_lambda': [1, 1.5, 3],
                         'scale_pos_weight': [1], 'subsample': [1]},
             verbose=1)

In [89]:
predicted_xgb = clf_grid_round2.predict(x_val)
print(classification_report(y_val, predicted_xgb))

              precision    recall  f1-score   support

           0       0.83      0.96      0.89     18403
           1       0.42      0.13      0.20      4105

    accuracy                           0.81     22508
   macro avg       0.63      0.55      0.55     22508
weighted avg       0.76      0.81      0.77     22508



In [90]:
predicted_xgb = clf_grid_round2.predict(xo_test)
print(classification_report(yo_test, predicted_xgb))

              precision    recall  f1-score   support

           0       0.83      0.96      0.89     18345
           1       0.43      0.14      0.21      4163

    accuracy                           0.81     22508
   macro avg       0.63      0.55      0.55     22508
weighted avg       0.76      0.81      0.76     22508



In [91]:
model_dictionary["xgb_grid_base_round2"]= average_precision_score(yo_test, predicted_xgb)

In [92]:
model_dictionary


{'modelxgb_base': 0.20839491452833558,
 'xgb_grid_base': 0.21728632479901608,
 'xgb_grid_base_round2': 0.2185626724133346}

In [93]:
previous_best = xgb_grid_base.best_params_

In [94]:
previous_best

{'eta': 0.01,
 'gamma': 1,
 'max_depth': 10,
 'reg_alpha': 1,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'subsample': 1}

Como se puede observar con estos resultados, el uso de nueva codificación y mayor regularización generaron que el modelo mejorar sus métricas de área bajo la curva PR. 

## 6. Oversampling SMOTE

<a href=https://arxiv.org/pdf/1106.1813.pdf>SMOTE</a> (Synthetic Minority Over-Sampling Technique) es un paper publicado el año 2002 en la revista Journal of Artificial Intelligence Research, el método propuesto se plantea como una opción para el método de undersampling o oversampling que se utiliza en muchos casos en la práctica cuando se tienen clases desbalanceadas. En específico, lo que hace este método es generar datos sintéticos de la clase minoritaria para así generar un oversampling de esta, en específico, el método genera muestras utilizando los k-vecinos más cercanos y realizando una perturbación de los features de la muestra en la dirección de dicho vecino. 

El método al hacer esta generación de datos sintéticos en la dirección de uno de sus vecinos más cercanos lo que hace es forzar la generalización de la zona de decisión de la clase minoritaria. Como hiper parpametro este algoritmo utiliza el porcentaje de oversampling que se desea realizar de esta. Además, en este trabajo utilizan una mezcla entre primero hacer uso del algoritmo SMOTE y luego realizar un undersampling sobre la clase mayoritaria. Para intentar mejorar dicho modelo haremos los dos métodos y veremos cual es mejor. 

### 6.1 SMOTE OverSample:
Para este caso se partirá realizando solamente un oversampling de la clase minoritaria, en específico, se buscará que dicha clase represente aproximadamente el 70% de los datos de la clase mayoritaria. Actualmente los datos de la clase **retraso** representan aproximadamente el 18% de la base de datos. Esto implica que realizaremos un aumento de 166% de los datos, esto con el fin de verificar si mediante el método SMOTE efectivamente generan mejoras considerables en los resultados encontrados en comparación al método basal.  

In [96]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
x_train, x_val, y_train, y_val = train_test_split(Xo, Yo, test_size = 0.33, random_state = 42)
x_train, x_val = column_trans.transform(x_train), column_trans.transform(x_val)
oversample = SMOTE(sampling_strategy=0.7, random_state=42, k_neighbors=10)
x_train, y_train = oversample.fit_resample(x_train, y_train)

In [97]:
print(f"Cantidad de vuelos {Y.count()}",f"Cantidad de atrasos {Y.sum()}", f"Porcentaje de vuelos atrasados en dataset:{Y.sum()/Y.count()}")
print(f"Cantidad de vuelos train {y_train.count()}", f"Cantidad de atrasos {y_train.sum()}", f"Porcentaje de vuelos atrasados en dataset:{y_train.sum()/y_train.count()}")
print(f"Cantidad de vuelos validation {y_val.count()}", f"Cantidad de atrasos {y_val.sum()}", f"Porcentaje de vuelos atrasados en dataset:{y_val.sum()/y_val.count()}")

Cantidad de vuelos 68206 Cantidad de atrasos 12614 Porcentaje de vuelos atrasados en dataset:0.18493974137172683
Cantidad de vuelos train 63221 Cantidad de atrasos 26032 Porcentaje de vuelos atrasados en dataset:0.4117619145537084
Cantidad de vuelos validation 22508 Cantidad de atrasos 4105 Porcentaje de vuelos atrasados en dataset:0.18237959836502576


In [99]:
modelxgb_smote = xgb.XGBClassifier(random_state=1,  objective= 'binary:logistic',  eval_metric='aucpr',**previous_best)
modelxgb_smote = modelxgb_smote.fit(x_train, y_train, eval_set=[(x_val, y_val)], verbose=False)
predicted_xgb = modelxgb_smote.predict(x_val)
print(classification_report(y_val, predicted_xgb))

              precision    recall  f1-score   support

           0       0.85      0.82      0.84     18403
           1       0.31      0.37      0.34      4105

    accuracy                           0.74     22508
   macro avg       0.58      0.60      0.59     22508
weighted avg       0.76      0.74      0.75     22508



In [100]:
predicted_xgb = modelxgb_smote.predict(xo_test)
print(classification_report(yo_test, predicted_xgb))

              precision    recall  f1-score   support

           0       0.85      0.82      0.84     18345
           1       0.33      0.39      0.36      4163

    accuracy                           0.74     22508
   macro avg       0.59      0.60      0.60     22508
weighted avg       0.76      0.74      0.75     22508



In [101]:
model_dictionary["xgb_smote"]= average_precision_score(yo_test, predicted_xgb)

In [102]:
model_dictionary

{'modelxgb_base': 0.20839491452833558,
 'xgb_grid_base': 0.21728632479901608,
 'xgb_grid_base_round2': 0.2185626724133346,
 'xgb_smote': 0.24054706513149243}

Una vez utilizado el algoritmo SMOTE se puede observar una mejora del 20% en los resultados obtenidos.

### 6.2 SMOTE y Random UnderSampling:

Otra manera de utilizar el método SMOTE es mediante el oversampling de la clase minoritaria para luego realizar un under sampling de la clase mayoritaria. Esto permite compensar un poco la pérdida de información por el under sampling mediante la generación de muestras sintéticas.

In [202]:
_, xo_test, _, yo_test = train_test_split(Xo, Yo, test_size = 0.33, random_state = 42, stratify=Yo)
x_train, x_val, y_train, y_val = train_test_split(Xo, Yo, test_size = 0.33, random_state = 42)
x_train, x_val, xo_test = column_trans.transform(x_train), column_trans.transform(x_val), column_trans.transform(xo_test)
oversample = SMOTE(sampling_strategy=0.5, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size = 0.33, random_state = 42)
x_train, y_train = oversample.fit_resample(x_train, y_train)
under = RandomUnderSampler(sampling_strategy=0.7, random_state=1)
x_train, y_train = under.fit_resample(x_train, y_train)


In [203]:
print(f"Cantidad de vuelos {Y.count()}",f"Cantidad de atrasos {Y.sum()}", f"Porcentaje de vuelos atrasados en dataset:{Y.sum()/Y.count()}")
print(f"Cantidad de vuelos train {y_train.count()}", f"Cantidad de atrasos {y_train.sum()}", f"Porcentaje de vuelos atrasados en dataset:{y_train.sum()/y_train.count()}")
print(f"Cantidad de vuelos train {y_val.count()}", f"Cantidad de atrasos {y_val.sum()}", f"Porcentaje de vuelos atrasados en dataset:{y_val.sum()/y_val.count()}")

Cantidad de vuelos 68206 Cantidad de atrasos 12614 Porcentaje de vuelos atrasados en dataset:0.18493974137172683
Cantidad de vuelos train 45156 Cantidad de atrasos 18594 Porcentaje de vuelos atrasados en dataset:0.4117725219239968
Cantidad de vuelos train 22508 Cantidad de atrasos 4105 Porcentaje de vuelos atrasados en dataset:0.18237959836502576


In [204]:
modelxgb_smote_under = xgb.XGBClassifier(random_state=1,  objective= 'binary:logistic', eval_metric='aucpr',  **previous_best)
modelxgb_smote_under = modelxgb_smote_under.fit(x_train, y_train, eval_set=[(x_val, y_val)], verbose=False)
predicted_xgb = modelxgb_smote_under.predict(x_val)
print(classification_report(y_val, predicted_xgb))

              precision    recall  f1-score   support

           0       0.86      0.81      0.83     18403
           1       0.31      0.38      0.35      4105

    accuracy                           0.73     22508
   macro avg       0.58      0.60      0.59     22508
weighted avg       0.76      0.73      0.74     22508



In [205]:
predicted_xgb = modelxgb_smote_under.predict(xo_test)
print(classification_report(yo_test, predicted_xgb))

              precision    recall  f1-score   support

           0       0.86      0.82      0.84     18345
           1       0.33      0.40      0.36      4163

    accuracy                           0.74     22508
   macro avg       0.59      0.61      0.60     22508
weighted avg       0.76      0.74      0.75     22508



In [208]:
model_dictionary["xgb_smote_under"]= average_precision_score(yo_test, predicted_xgb)

In [209]:
model_dictionary

{'modelxgb_base': 0.20839491452833558,
 'xgb_grid_base': 0.21728632479901608,
 'xgb_grid_base_round2': 0.2185626724133346,
 'xgb_smote': 0.24054706513149243,
 'xgb_smote_under': 0.24140623473961104,
 'xgb_grid_smote': 0.2415843165271429,
 'more_features': 0.2376063877394071,
 'xgb_important_features': 0.23492118067480844}

## 7. Selección Mejor par Modelo-Método de Over-Under Sampling:

Como se indicó anteriormente, la manera en que se definirá el mejor modelo para el set de datos generado es mediante la métrica de AUCPR, dicha métrica nos permite encontrar el mejor modelo que considere tanto el recall como la precisión. De esta manera, el modelo elegido es el uso de SMOTE sin under sampling.

### 7.1 Grid Search Mejor Modelo:

In [136]:
x_train, x_val, y_train, y_val = train_test_split(Xo, Yo, test_size = 0.33, random_state = 42)
x_train, x_val = column_trans.transform(x_train), column_trans.transform(x_val)
oversample = SMOTE(sampling_strategy=0.7, random_state=42, k_neighbors=10)
x_train, y_train = oversample.fit_resample(x_train, y_train)

In [137]:
from sklearn.model_selection import GridSearchCV
modelxgb_smote_grid = xgb.XGBClassifier(random_state=1,  objective= 'binary:logistic',  eval_metric='aucpr', **previous_best)
parameters = { "reg_lambda": [0, 0.1, 1],
                "scale_pos_weight": [1, 20, 30,  1000],
                "subsample": [0.5, 1],
                "eta": [0.01, 0.1, 1],
                "gamma": [0, 0.25, 1]}
xgb_smote_grid = GridSearchCV(modelxgb_grid, param_grid=parameters,
    cv =2, n_jobs=-1, verbose=1)

In [138]:
xgb_smote_grid.fit(x_train, y_train, eval_set=[(x_val, y_val)], verbose=False)

Fitting 2 folds for each of 216 candidates, totalling 432 fits


GridSearchCV(cv=2,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'eta': [0.01, 0.1, 1], 'gamma': [0, 0.25, 1],
                         'reg_lambda': [0, 0.1, 1],
                         'scale_pos_weight': [1, 20, 30, 1000],
                         'subsample': [0.5, 1]},
             verbose=1)

In [139]:
predicted_xgb = xgb_smote_grid.predict(x_val)
print(classification_report(y_val, predicted_xgb))

              precision    recall  f1-score   support

           0       0.86      0.81      0.83     18403
           1       0.31      0.39      0.35      4105

    accuracy                           0.73     22508
   macro avg       0.59      0.60      0.59     22508
weighted avg       0.76      0.73      0.74     22508



In [140]:
predicted_xgb = xgb_smote_grid.predict(xo_test)
print(classification_report(yo_test, predicted_xgb))

              precision    recall  f1-score   support

           0       0.86      0.81      0.83     18345
           1       0.33      0.40      0.36      4163

    accuracy                           0.74     22508
   macro avg       0.59      0.61      0.60     22508
weighted avg       0.76      0.74      0.75     22508



In [141]:
xgb_smote_grid.best_params_

{'eta': 1, 'gamma': 0, 'reg_lambda': 0, 'scale_pos_weight': 1, 'subsample': 1}

In [142]:
model_dictionary["xgb_grid_smote"]= average_precision_score(yo_test, predicted_xgb)

In [143]:
model_dictionary

{'modelxgb_base': 0.20839491452833558,
 'xgb_grid_base': 0.21728632479901608,
 'xgb_grid_base_round2': 0.2185626724133346,
 'xgb_smote': 0.24054706513149243,
 'xgb_smote_under': 0.2284234527919285,
 'xgb_grid_smote': 0.2415843165271429}

In [ ]:
previous_best = xgb_smote_grid.best_params_

De este modo, viendo el diccionario de valores, se puede ver que el mejor valor obtenido es el con xgb_grid_smote

## 8. Entrenando modelo agregando features:

In [155]:
COLUMNAS_MAS_FEATURES = ['OPERA', 'MES', 'TIPOVUELO', 'SIGLADES', "SIGLAORI",'DIANOM']
data_to_pipe = data[COLUMNAS_MAS_FEATURES]
Xo, Yo = data_to_pipe, data["atraso_15"]
_, xo_test, _, yo_test = train_test_split(Xo, Yo, test_size = 0.33, random_state = 42, stratify=Yo)
# generamos una codificaciones ordinaria para los datos: 
column_trans = make_column_transformer((OrdinalEncoder(), COLUMNAS_MAS_FEATURES))
# ajustamos la codificacion
column_trans.fit(data_to_pipe)
xo_test = column_trans.transform(xo_test)

# generamos una funcion que genere pipelines: 
Makepipe = lambda the_model: make_pipeline(column_trans, the_model)

In [156]:
x_train, x_val, y_train, y_val = train_test_split(Xo, Yo, test_size = 0.33, random_state = 42)
x_train, x_val = column_trans.transform(x_train), column_trans.transform(x_val)
oversample = SMOTE(sampling_strategy=0.7, random_state=42, k_neighbors=10)
x_train, y_train = oversample.fit_resample(x_train, y_train)

In [157]:
modelxgb = xgb.XGBClassifier(random_state=1, objective= 'binary:logistic',  eval_metric='aucpr', **previous_best)
modelxgb = modelxgb.fit(x_train, y_train, eval_set=[(x_val, y_val)], verbose=False)
predicted_xgb = modelxgb.predict(x_val)
print(classification_report(y_val, predicted_xgb))

              precision    recall  f1-score   support

           0       0.84      0.94      0.88     18403
           1       0.40      0.18      0.25      4105

    accuracy                           0.80     22508
   macro avg       0.62      0.56      0.57     22508
weighted avg       0.76      0.80      0.77     22508



In [158]:
predicted_xgb = modelxgb.predict(xo_test)
print(classification_report(yo_test, predicted_xgb))

              precision    recall  f1-score   support

           0       0.84      0.94      0.89     18345
           1       0.44      0.21      0.28      4163

    accuracy                           0.80     22508
   macro avg       0.64      0.57      0.58     22508
weighted avg       0.77      0.80      0.77     22508



In [160]:
model_dictionary["more_features"] = average_precision_score(yo_test, predicted_xgb)

In [164]:
model_dictionary

{'modelxgb_base': 0.20839491452833558,
 'xgb_grid_base': 0.21728632479901608,
 'xgb_grid_base_round2': 0.2185626724133346,
 'xgb_smote': 0.24054706513149243,
 'xgb_smote_under': 0.2284234527919285,
 'xgb_grid_smote': 0.2415843165271429,
 'more_features': 0.2376063877394071}

## 9. Entrenando modelo solo con features importantes:

In [170]:
feature_important = xgb_smote_grid.best_estimator_.get_booster().get_score(importance_type='weight')
keys = list(feature_important.keys())
values = list(feature_important.values())

important_features = pd.DataFrame(data=values, index=keys, columns=["score"]).sort_values(by = "score", ascending=False)

In [178]:
X = Xo.iloc[:, [0, 2]]

In [184]:
important_features.head()

,score
f2,1707.0
f0,1399.0
f1,117.0


In [181]:
# obtenemos los datos del dataframe:
data_to_pipe = data[["OPERA", "MES"]]

# volvemos a generar los datos originales con la nueva codificacion:
Xo, Yo = data_to_pipe, data["atraso_15"]
_, xo_test, _, yo_test = train_test_split(Xo, Yo, test_size = 0.33, random_state = 42, stratify=Yo)

In [182]:
# generamos una codificaciones ordinaria para los datos: 
column_trans = make_column_transformer((OrdinalEncoder(), ["OPERA", "MES"]))
# ajustamos la codificacion
column_trans.fit(data_to_pipe)
xo_test = column_trans.transform(xo_test)

In [187]:
x_train, x_val, y_train, y_val = train_test_split(Xo, Yo, test_size = 0.33, random_state = 42)
x_train, x_val = column_trans.transform(x_train), column_trans.transform(x_val)
oversample = SMOTE(sampling_strategy=0.7, random_state=1)
x_train, y_train = oversample.fit_resample(x_train, y_train)

In [188]:
modelxgb = xgb.XGBClassifier(random_state=1, objective= 'binary:logistic',   eval_metric='aucpr', **previous_best)
modelxgb = modelxgb.fit(x_train, y_train, eval_set=[(x_val, y_val)], verbose=False)
predicted_xgb = modelxgb.predict(x_val)
print(classification_report(predicted_xgb, y_val))

              precision    recall  f1-score   support

           0       0.87      0.85      0.86     19016
           1       0.29      0.34      0.31      3492

    accuracy                           0.77     22508
   macro avg       0.58      0.59      0.58     22508
weighted avg       0.78      0.77      0.77     22508



In [189]:
predicted_xgb = modelxgb.predict(xo_test)

In [191]:
model_dictionary["xgb_important_features"] = average_precision_score(yo_test, predicted_xgb)

## 10. Selección Mejor Modelo

Observando los resultados encontrados bajo todas las pruebas previamente hechas, se puede notar que el mejor modelo encontrado fue el que utiliza SMOTE como método de oversampling y luego fue optimizado por grid search. A continuación mostramos los resultados y el valor de cross_validation que presenta:

In [193]:
# obtenemos los datos del dataframe:
data_to_pipe = data[COLUMNAS_BASALES]

# volvemos a generar los datos originales con la nueva codificacion:
Xo, Yo = data_to_pipe, data["atraso_15"]
_, xo_test, _, yo_test = train_test_split(Xo, Yo, test_size = 0.33, random_state = 42, stratify=Yo)

In [194]:
# generamos una codificaciones ordinaria para los datos: 
column_trans = make_column_transformer((OrdinalEncoder(), COLUMNAS_BASALES))
# ajustamos la codificacion
column_trans.fit(data_to_pipe)
xo_test = column_trans.transform(xo_test)

In [195]:
x_train, x_val, y_train, y_val = train_test_split(Xo, Yo, test_size = 0.33, random_state = 42)
x_train, x_val = column_trans.transform(x_train), column_trans.transform(x_val)
oversample = SMOTE(sampling_strategy=0.7, random_state=42, k_neighbors=10)
x_train, y_train = oversample.fit_resample(x_train, y_train)

In [192]:
model_dictionary

{'modelxgb_base': 0.20839491452833558,
 'xgb_grid_base': 0.21728632479901608,
 'xgb_grid_base_round2': 0.2185626724133346,
 'xgb_smote': 0.24054706513149243,
 'xgb_smote_under': 0.2284234527919285,
 'xgb_grid_smote': 0.2415843165271429,
 'more_features': 0.2376063877394071,
 'xgb_important_features': 0.23492118067480844}

In [196]:
from sklearn.model_selection import RepeatedStratifiedKFold
kfolds = RepeatedStratifiedKFold(n_splits=5, random_state=32)
kfolds.get_n_splits(Xo, Yo)
X, Y = column_trans.transform(Xo), Yo
result = cross_val_score(modelxgb, xo_test, yo_test, cv=kfolds, n_jobs=-1, scoring='average_precision')
print(f"AUCPR result: {result.mean()} +- {result.std()}" )

AUCPR result: 0.3116357724170816 +- 0.010272190469080204


In [198]:
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder
column_trans = make_column_transformer((OrdinalEncoder(), COLUMNAS_BASALES))
column_trans.fit(data_to_pipe)
pipe = make_pipeline(column_trans, xgb_smote_grid.best_estimator_)

In [201]:
import joblib
model_name = "backend/production_model.sav"
joblib.dump(pipe, model_name)

['backend/production_model.sav']

A través de esta pregunta se pudo mostrar que fue posible mejorar el modelo generado más del 20%, dentro de las posibles mejoras que aún podrían hacerse se encuentra la posibilidad de agregar las distancias entre ciudad de origen y ciudad de destino, para que así el modelo tenga una referencia de que tan lejos quedan las ciudades y por ende pueda considerar esto en su predicción.

## 11. API:
# Pregunta 3: 
Habilita el modelo seleccionado como API REST para ser expuesto

Para resolver este problema lo que se hizo en primera instancia fue generar un proyecto de Django. En dicho proyecto se generaró una api que contiene los métodos post y get, en específico el método post es el que se encarga de generar las predicciones para el equipo y es el que se encuentra a disposición para realizar las consultas. El método post debe realizarse con la siguiente url http://localhost:8000/api/estaAtrasado 

Dicha uri debe ser utilizada con un body que debe contener los siguiente: 
- OPERA
- TIPOVUELO
- MES
- VLO_I

los primeros tres elementos de la lista corresponden a los features utilizados por el modelo, mientras que VLO_I se utiliza como indicador del vuelo que se está buscando predecir si llegará con retraso. De esta manera, el equipo debe mandar el post request con los valores que corresponden a la base de datos, ya que el modelo de pipeline realizado es el que posteriormente se encarga de transformar los valores. Otro punto importante se encuentra en que la api generada puede trabajar con múltiples vuelos al mismo tiempo, esto se debe a que el api está hecha para poder mandar un arreglo de json (donde cada uno de ellos representa un vuelo), por lo que el payload final del método rest es el siguiente: 

{"vuelos": [{"MES": 3,"TIPOVUELO": "N", "OPERA": "K.L.M.", "VLO_I": 226}, {"MES": 12,"TIPOVUELO": "I", "OPERA": "Air Canada", "VLO_I":226}]}


Además se realizó un método GET, dicho método trae el historial de consultas que se le han realizado al servidor, este utiliza la misma api que el método anterior. 


## Pregunta 4:
Haz pruebas de estrés a la API con el modelo expuesto con al menos 50.000 requests durante 45 segundos.
Para esto debes utilizar esta herramienta y presentar las métricas obtenidas.

Cargamos en primera instancia la imagen de wrk desde docker: 

In [ ]:
! docker pull williamyeh/wrk

Hacemos un build del servidor desde docker:

In [11]:
! docker-compose build

Building web
[+] Building 0.0s (0/1)                                                         
[+] Building 0.1s (2/2)                                                         
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 32B                                        0.0s
 => [internal] load .dockerignore                                          0.1s
 => => transferring context: 2B                                            0.0s
[+] Building 0.3s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 32B                                        0.0s
 => [internal] load .dockerignore                                          0.1s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.8              0.1s
[+] Building 0.4s (2/3) 

Levantamos el servidor en docker. 

In [12]:
! docker-compose up -d

Creating network "neuralworkschallenge_default" with the default driver
Creating neuralworkschallenge_web_1 ... 
ting neuralworkschallenge_web_1 ... done

### 11.1 POST Method: 

A modo de muestras se agrega el script que se utilizará para realizar las pruebas de estrés. 

%% Inicio %%

wrk.method = "POST"

wrk.body = '{"vuelos": [{"MES": 3,"TIPOVUELO": "N", "OPERA": "K.L.M.", "VLO_I": 226}, {"MES": 12,"TIPOVUELO": "I", "OPERA": "Air Canada", "VLO_I":226}]}'

wrk.headers["Content-Type"] = "application/json"

In [10]:
! docker run --net="host" --rm -v `pwd`/scripts:/scripts williamyeh/wrk -t10 -c50000 -d45s -s /scripts/post_test.lua http://host.docker.internal:8000/api/estaAtrasado

Running 45s test @ http://host.docker.internal:8000/api/estaAtrasado
  10 threads and 50000 connections
  Thread Stats   Avg      Stdev     Max   +/- Stdev
    Latency     0.00us    0.00us   0.00us    -nan%
    Req/Sec     0.00      0.00     0.00      -nan%
  0 requests in 1.83m, 0.00B read
  Socket errors: connect 21637, read 0, write 305, timeout 0
Requests/sec:      0.00
Transfer/sec:       0.00B


### 11.1 Método Get: 

In [5]:
! docker run --net="host" --rm -v scripts:/scripts williamyeh/wrk -t10000 -c50000 -d45s http://host.docker.internal:8000/api/estaAtrasado

Running 45s test @ http://host.docker.internal:8000/api/estaAtrasado
  100 threads and 50000 connections
  Thread Stats   Avg      Stdev     Max   +/- Stdev
    Latency     0.00us    0.00us   0.00us    -nan%
    Req/Sec     0.00      0.00     0.00      -nan%
  0 requests in 1.01m, 1.43KB read
  Socket errors: connect 20906, read 0, write 817, timeout 0
Requests/sec:      0.00
Transfer/sec:      24.09B


## 12. Propuesta de Mejora

Los resultados anteriores muestran que la api desarrollada tiene problemas al momento de trabajar con las 50000 conexiones, en específico se cree que el problema más que en la api en específico se debe a la configuración del servidor y a los recursos disponibles, ya que este se queda sin recursos al correr el test. De este modo como propuesta de mejora, se propone utilizar el caché en los request del servidor que sean muy frecuentes de manera de utilizar dichos resultados y no utilizar recursos con casos en que ya se tiene los resultados, esto sería especialmente útil en caso de un avión que ya haya sido ingresado y por ende no tenga necesidad de ingresarse nuevamente como consulta. 

Además, se cree que ante la gran cantidad de request que llegan según lo testeado anteriormente, se podría utilizar computación distributiva de manera de permitir que la carga total de las consultas no esté centrada en un solo computador, sino que en varios de manera de simplificar la carga. Además es posible utilizar aplicaciones cloud que están justamente planteadas como soluciones para los casos de obtención masiva de datos. 

Por último, otra posible mejora a la api propuesta es la utilización de apis asíncronas, las cuales permitan que las consultas sean recibidas por parte del servidor pero no sea necesario mantener una sesión abierta con el cliente. De esta manera se podría bajar la carga que generan conexiones abiertas. 